In [1]:
# General logic is to create two dataframes:
# 1 with all monthly account arr values
# 2 with all monthly account renewal values

# df_1 is supplied first with a diff column for each month which takes the differences month over month and appends it in its own separate column
# df_2 is appended to df_1
# a dictionary comprehension is used to create all the df_month

# filtering is done on each df_month
# each df_month is output into its own sheet in one file


In [2]:
import pandas as pd
from datetime import date,datetime
from pandas.tseries.offsets import DateOffset, MonthEnd, MonthBegin
import numpy as np

In [3]:
# df = pd.read_excel('/Users/jesperfriislarnaes/Library/CloudStorage/OneDrive-eloomi/Scripts/Shared_Scripts/GRR_Report/test-data.xlsx')
df = pd.read_excel('/Users/emirkabasoglu/Desktop/Board_Metrics_Input_File.xlsx')

In [4]:
# report_period_start = pd.Timestamp(2021,10,1,0)
# report_period_end = report_period_start + DateOffset(months=36)
report_period_start = pd.Timestamp(2024,7,1,0)
report_period_end = report_period_start + DateOffset(months=13)

In [5]:
# function to loop over months in period and return subscription values in those months
revenue_periods = []
def get_revenue_periods(df,start,end):
    df_out = df.copy()

    # set up while loop to go over months
    while start < end:
        revenue_date = start + MonthEnd(0)
        #print(revenue_date)
        if df_out.empty:
            df_out[f'{start}'] = np.nan
        else:
            # get arr value of subscription in period            
            def get_sub_arr_value(sub_start, sub_end, sub_amount, cancelled):
                if sub_start <= revenue_date and sub_end >= revenue_date:
                    return sub_amount
                elif sub_end < revenue_date and cancelled != 'y':
                    return sub_amount

            df_out[f'{pd.to_datetime(revenue_date).strftime("%y-%b")}'] = df_out.apply(lambda x: get_sub_arr_value(x['start'], x['end'], x['value'], x['cancelled']), axis=1).replace(np.nan, 0)
       
        start = start + DateOffset(months=1)
        revenue_periods.append(revenue_date)
    
    return df_out

In [6]:
# run function to get revenue periods
df_revenue = get_revenue_periods(df,report_period_start,report_period_end)

In [7]:
# aggregate revenue period values on ids and product type
# print(starting_col_position)
rev_starting_col_position = df_revenue.shape[1] - len(revenue_periods)

# print(revenue_cols)
revenue_cols = list(df_revenue.iloc[:,rev_starting_col_position:])

# group by id and sum revenue periods on id level
df_revenue_grp = df_revenue.groupby(['id','product'])[revenue_cols].sum()

In [8]:
# group by id and sum revenue periods on id level
df_total_revenue_grp = df_revenue.groupby(['id'])[revenue_cols].sum()

In [9]:
df_total_revenue_grp

,24-Jul,24-Aug,24-Sep,24-Oct,24-Nov,24-Dec,25-Jan,25-Feb,25-Mar,25-Apr,25-May,25-Jun,25-Jul
id,,,,,,,,,,,,,
225,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1010,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1030,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
1236,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
2623,8000.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000,7200.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...
989214071,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000,0.00000
996756246,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000,21042.56000
996756248,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715,19394.83715


In [10]:
# function to loop over months in period and return subscription renewal values in those months
renewal_periods = []
def get_renewal_periods(df,start,end):
    df_out = df.copy()
    # set up while loop to go over months
         
    while start < end:
        renewal_date = start + MonthEnd(0)
        renewal_period_start = start + DateOffset(days=-1)
        renewal_period_end = renewal_date + DateOffset(days=-1)

        if df_out.empty:
            df[f'{start}'] = np.nan

        else:
            def get_sub_renewal_value(sub_end, sub_amount, cancelled, sub_start):
                if sub_end >= renewal_period_start and sub_end <= renewal_period_end:
                    return sub_amount
                #elif sub_contract_term_start > sub_start:
                    #if sub_contract_term_start + DateOffset(days=-1) >= renewal_period_start and sub_contract_term_start + DateOffset(days=-1) <= renewal_period_end:
                        #return sub_amount       

            df_out[f'{pd.to_datetime(renewal_date).strftime("%y-%b")}'] = df_out.apply(lambda x: get_sub_renewal_value(x['end'], x['value'], x['cancelled'],  x['start']), axis=1).replace(np.nan, 0) 

        start = start + DateOffset(months=1)
        renewal_periods.append(renewal_date)
    
    return df_out

# todo maybe add logic to capture all prior periods and all future ones

In [11]:
# run function to get renewal periods
df_renewal = get_renewal_periods(df,report_period_start,report_period_end)

In [12]:
# aggregate renewal period values on ids
# print(starting_col_position)
ren_starting_col_position = df_renewal.shape[1] - len(renewal_periods)
# print(revenue_cols)
renewal_cols = list(df_renewal.iloc[:,ren_starting_col_position:])
# group by id and sum renewal periods on id and product type level
df_renewal_grp = df_renewal.groupby(['id','product'])[renewal_cols].sum()

In [13]:
# get month-over-month diffs and append these at axis=1
df_revenue_diffs_renewals = pd.concat(
    [df_revenue_grp, 
    df_revenue_grp.diff(axis=1).iloc[:, 1:].add_suffix('-Diff'),
    df_renewal_grp.iloc[:,1:].add_suffix('-Renewal')], axis=1)

In [14]:
# set up dictionary of dataframes for each period named according to the column period names
d_of_dfs = {b: df_revenue_diffs_renewals.filter(regex=rf'^{a}$|{b}') for a, b in zip(df_revenue_grp.columns, df_revenue_grp.columns[1:])}

In [15]:
d_of_dfs

{'24-Aug':                          24-Jul       24-Aug  24-Aug-Diff  24-Aug-Renewal
 id        product                                                        
 225       software      0.00000      0.00000          0.0             0.0
 1010      software      0.00000      0.00000          0.0             0.0
 1030      software      0.00000      0.00000          0.0             0.0
 1236      software      0.00000      0.00000          0.0             0.0
 2623      software   8000.00000   7200.00000       -800.0          8000.0
 ...                         ...          ...          ...             ...
 996756248 content       0.00000      0.00000          0.0             0.0
           software  19394.83715  19394.83715          0.0             0.0
 996756261 software   7132.73000   7132.73000          0.0             0.0
 996756270 content       0.00000      0.00000          0.0             0.0
           software   3556.30000   3556.30000          0.0             0.0
 
 [1964 rows x

In [16]:
for name, df in d_of_dfs.items():
   df['month'] = name

<ipython-input-16-20cd3bdaffda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = name
<ipython-input-16-20cd3bdaffda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['month'] = name
<ipython-input-16-20cd3bdaffda>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ver

In [17]:
# transform each df in the dictionary of dfs

for name, dfs in d_of_dfs.items():
    # Set the index of d_of_dfs[name] to 'id'
    if 'id' in d_of_dfs[name].columns and 'id' in dfs.columns:
        d_of_dfs[name] = d_of_dfs[name].set_index('id')
    
    # Drop duplicates in df_main and set its index to 'id'
        df_main_unique = d_of_dfs.drop_duplicates(subset=['customer_name']).set_index('id')
    
    # Perform the join operation
        d_of_dfs[name] = d_of_dfs[name].join(df_main_unique['customer_name']).reset_index()

    # Drop duplicates in the main df and set index to 'id'
    
    #d_of_dfs[name] = d_of_dfs[name].join(df.drop_duplicates(subset=['customer_name']).set_index('id')['customer_name'])

    #changing names of columns and rearranging columns to have customer name after id
    new_col_names = ['Prior Month', 'Current Month', 'M-o-M Change', 'Up for Renewal', 'Customer']
    new_col_order = ['Customer', 'Prior Month', 'Current Month', 'M-o-M Change', 'Up for Renewal']
    d_of_dfs[name] = d_of_dfs[name].set_axis(new_col_names, axis='columns', copy=False)[new_col_order]
    
    #filtering out customers with no changes m-o-m or subscriptions up for renewal or new logos
    d_of_dfs[name] = d_of_dfs[name][(d_of_dfs[name]['M-o-M Change'] != 0) | (d_of_dfs[name]['Up for Renewal'] != 0)]
    d_of_dfs[name] = d_of_dfs[name][(d_of_dfs[name]['Prior Month'] != 0)]

    #rounding values
    d_of_dfs[name] = d_of_dfs[name].round(0)
    
    #adding column with value of renewed
    def calc_renewed(mom_change, up_for_renewal):
        if up_for_renewal == 0:
            return 0
        else:
            return up_for_renewal + mom_change

    d_of_dfs[name]['Renewed'] = d_of_dfs[name].apply(lambda x: calc_renewed(x['M-o-M Change'], x['Up for Renewal']), axis=1)

    #adding column for signifying type of change on the account
    def account_change(up_for_renewal, mom_change, current_month):
        if up_for_renewal > 0 and mom_change < 0 and current_month > 0:
            return 'Renewed With Downsell'
        elif up_for_renewal > 0 and mom_change > 0:
            return 'Renewed With Upsell'
        elif up_for_renewal > 0 and current_month == 0:
            return 'Churn'
        elif up_for_renewal == 0 and mom_change > 0:
            return 'Out-of-Period Upsell'
        elif up_for_renewal == 0 and mom_change < 0:
            return 'Out-of-Period Downsell'
        elif up_for_renewal > 0 and mom_change == 0:
            return 'Renewed with No Change'

    d_of_dfs[name]['Account Change Type'] = d_of_dfs[name].apply(lambda x: account_change(x['Up for Renewal'], x['M-o-M Change'], x['Current Month']), axis=1)
    
    #change name of axis 0
    d_of_dfs[name].rename_axis(['id','product'], inplace=True)


In [18]:
#for name, df in d_of_dfs.items():
   #df['month'] = name

In [19]:
# Concatenate the DataFrames into a single DataFrame
concatenated_df = pd.concat(d_of_dfs.values())
concatenated_df = concatenated_df.reset_index()


In [20]:
concatenated_df.to_excel('Renewal_Report_by_Product.xlsx')

In [21]:
# logic for writing each df in the dictionary to its own sheet in the same Excel file
#with pd.ExcelWriter('Renewal_Report_by_Product.xlsx') as writer:
    #for name, df in d_of_dfs.items():
        #d_of_dfs[name].to_excel(writer, sheet_name=f'{name}')

In [22]:
#---=== END ===---